# Environment setting / 環境設定
Best practices: Multi-Timestamp data - Time Anchoring / 最佳實踐：多時間點資料 - 時間定錨

In [1]:
import os
import sys
from pathlib import Path

# 自動載入 utils / Auto-load utils
if "COLAB_GPU" in os.environ:
    url = "https://raw.githubusercontent.com/nics-tw/petsard/main/demo/utils.py"
    exec(open(url).read())
else:
    # 靜默搜尋 utils.py / Silent search for utils.py
    current = Path.cwd()
    for _ in range(5):
        if (current / "utils.py").exists():
            sys.path.insert(0, str(current))
            break
        current = current.parent

    # 匯入 utils 模組 / Import utils module
    from utils import quick_setup

# 快速設定 / Quick setup
is_colab, branch, yaml_path = quick_setup(
    yaml_file="multi-timestamp.yaml",
    benchmark_data=[
        "best-practices_multi-table",
    ],
    branch="main",  # 可選，預設為 "main"
)

from petsard import Executor

🚀 PETsARD v1.5.1
📅 2025-07-31 14:33:37 UTC+8
📁 Subfolder: best-practices
📄 YAML path: petsard/demo/best-practices/multi-timestamp.yaml
⚙️ Configuration content:
---
Loader:
  data:
    filepath: 'benchmark/best-practices_multi-table.csv'
Preprocessor:
  time-anchoring:
    method: 'default'
    config:
      scaler:
        'established_date':
          # 以公司成立日為錨點，計算與申請、核准、追蹤等重要時間點的天數差異
          # Using company establishment date as anchor to calculate day differences
          #  with application, approval and tracking dates
          method: 'scaler_timeanchor'
          reference:
            - 'apply_date'
            - 'approval_date'
            - 'tracking_date_last_tracking_date'
          unit: 'D' # D 代表以天為單位 D represents measurement in days
Synthesizer:
  demo:
    method: 'default'
Postprocessor:
  demo:
    method: 'default'
Reporter:
  output:
    method: 'save_data'
    source: 'Postprocessor'
...


In [2]:
from pprint import pprint

import pandas as pd


multitable = pd.read_csv("benchmark/best-practices_multi-table.csv")

print("Multi-table columns: ")
pprint(multitable.columns.tolist())

Multi-table columns: 
['company_id',
 'industry',
 'sub_industry',
 'city',
 'district',
 'established_date',
 'capital',
 'first_apply_application_id',
 'first_apply_loan_type',
 'first_apply_apply_date',
 'first_apply_approval_date',
 'first_apply_status',
 'first_apply_amount_requested',
 'first_apply_amount_approved',
 'latest_apply_application_id',
 'latest_apply_loan_type',
 'latest_apply_apply_date',
 'latest_apply_approval_date',
 'latest_apply_status',
 'latest_apply_amount_requested',
 'latest_apply_amount_approved',
 'latest_track_application_id',
 'latest_track_profit_ratio_avg_profit_ratio',
 'latest_track_profit_ratio_min_profit_ratio',
 'latest_track_profit_ratio_profit_ratio_std',
 'latest_track_profit_ratio_negative_profit_count',
 'latest_track_tracking_date_tracking_months',
 'latest_track_tracking_date_last_tracking_date',
 'latest_track_revenue_avg_revenue',
 'latest_track_revenue_revenue_growth',
 'latest_track_risk_level_last_risk',
 'latest_track_risk_level_seco

In [3]:
multitable.head(5).T

,0,1,2,3,4
company_id,C000001,C000002,C000003,C000004,C000005
industry,營建工程,營建工程,製造業,營建工程,批發零售
sub_industry,環保工程,建築工程,金屬加工,環保工程,零售
city,新北市,臺北市,臺北市,桃園市,臺北市
district,板橋區,內湖區,內湖區,中壢區,內湖區
established_date,2019-11-03,2017-01-02,2012-05-29,2010-09-24,2010-07-24
capital,19899000,17359000,5452000,20497000,17379000
first_apply_application_id,A00000001,A00000006,A00000008,NaN,A00000014
first_apply_loan_type,廠房擴充,數位轉型,廠房擴充,NaN,疫後紓困
first_apply_apply_date,2022-01-21,2020-03-17,2016-05-08,NaN,2014-01-04


# Execution and Result / 執行與結果

In [4]:
exec_case = Executor(config=yaml_path)
exec_case.run()

Now is petsard_Loader[data]_Preprocessor[time-anchoring]_Synthesizer[demo]_Postprocessor[demo] save to csv...


In [5]:
exec_case.get_result()[
    "Loader[data]_Preprocessor[time-anchoring]_Synthesizer[demo]_Postprocessor[demo]_Reporter[output]"
][
    "Loader[data]_Preprocessor[time-anchoring]_Synthesizer[demo]_Postprocessor[demo]"
].head(5).T

,0,1,2,3,4
company_id,C000849,C000830,C000085,C000464,C000715
industry,服務業,批發零售,營建工程,製造業,服務業
sub_industry,零售,紡織,民生用品,室內裝修,建築工程
city,桃園市,臺北市,臺北市,桃園市,新北市
district,龜山區,信義區,三重區,新莊區,龜山區
established_date,2018-06-15 00:00:00,2014-09-28 00:00:00,2013-04-01 00:00:00,2013-04-14 00:00:00,2010-09-22 00:00:00
capital,24407916.538528,39348536.806233,10471493.161117,4760511.00849,38722286.87814
first_apply_application_id,A00002110,NaN,A00000233,A00001177,A00001786
first_apply_loan_type,數位轉型,NaN,數位轉型,數位轉型,營運週轉金
first_apply_apply_date,2013-05-20 00:00:00,NaN,2013-03-31 00:00:00,2012-01-26 00:00:00,2014-06-03 00:00:00
